In [66]:
from hydradx.model.amm.omnipool_amm import OmnipoolState
from hydradx.model.amm.agents import Agent
from hydradx.model.processing import parse_kraken_orderbook
from hydradx.model.amm.arbitrage_agent import get_arb_swaps, calculate_profit, execute_arb
from hydradx.model.amm.centralized_market import CentralizedMarket, OrderBook
from hydradxapi import HydraDX
from pprint import pprint
import requests



import json, os


In [67]:

resp = requests.get('https://api.kraken.com/0/public/Depth?pair=HDXUSD')
y = resp.json()
resp = requests.get('https://api.kraken.com/0/public/Depth?pair=DOTUSD')
z = resp.json()

# f = open('data/hdxusd_orderbook.json', 'r')
# y = json.loads(f.read())
# f = open('data/dotusd_orderbook.json', 'r')
# z = json.loads(f.read())

hdxusd_orderbook = y['result']['HDXUSD']
dotusd_orderbook = z['result']['DOTUSD']

pprint(hdxusd_orderbook)



{'asks': [['0.00440000', '100000.000', 1698068392],
          ['0.00460000', '74266.195', 1698077455],
          ['0.00470000', '51945.220', 1698074459],
          ['0.00480000', '787191.668', 1698074459],
          ['0.00500000', '910153.391', 1698067667],
          ['0.00510000', '50817.378', 1698074459],
          ['0.00520000', '613537.854', 1698071356],
          ['0.00540000', '50066.750', 1698075960],
          ['0.00550000', '778181.149', 1698069297],
          ['0.00570000', '600000.000', 1697287503],
          ['0.00590000', '104245.755', 1697282989],
          ['0.00600000', '54839.984', 1698047402],
          ['0.00610000', '693107.172', 1698047402],
          ['0.00640000', '5993.643', 1697192400],
          ['0.00660000', '84489.798', 1697615971],
          ['0.00700000', '2601.121', 1695280194],
          ['0.00730000', '6718.580', 1693667982],
          ['0.00740000', '12499.639', 1693642144],
          ['0.00750000', '18333.334', 1697662593],
          ['0.00800000', '

In [68]:
RPC="wss://hydradx-rpc.dwellir.com"

chain = HydraDX(RPC) 
chain.connect()

In [69]:
# Omnipool metadata
metadata = [
    {'symbol': 'HDX', 'id': 0, 'decimals': 12},
    {'symbol': 'LRNA', 'id': 1, 'decimals': 12},
    {'symbol': 'DAI', 'id': 2, 'decimals': 18},
    {'symbol': 'WBTC', 'id': 3, 'decimals': 8},
    {'symbol': 'WETH', 'id': 4, 'decimals': 18},
    {'symbol': 'DOT', 'id': 5, 'decimals': 10},
    {'symbol': 'APE', 'id': 6, 'decimals': 18},
    {'symbol': 'USDC', 'id': 7, 'decimals': 6},
    {'symbol': 'PHA', 'id': 8, 'decimals': 12},
    {'symbol': 'ASTR', 'id': 9, 'decimals': 18},
    {'symbol': 'USDT', 'id': 10, 'decimals': 6},
    {'symbol': 'IBTC', 'id': 11, 'decimals': 8},
    {'symbol': 'ZTG', 'id': 12, 'decimals': 10},
    {'symbol': 'CFG', 'id': 13, 'decimals': 18},
    {'symbol': 'BNC', 'id': 14, 'decimals': 12},
    {'symbol': 'vDOT', 'id': 15, 'decimals': 10},
    {'symbol': 'GLMR', 'id': 16, 'decimals': 18},
    {'symbol': 'INTR', 'id': 17, 'decimals': 10},
    {'symbol': 'DAI', 'id': 18, 'decimals': 18},
    {'symbol': 'WBTC', 'id': 19, 'decimals': 8},
    {'symbol': 'WETH', 'id': 20, 'decimals': 18},
    {'symbol': 'USDC', 'id': 21, 'decimals': 6},
    {'symbol': 'USDC', 'id': 22, 'decimals': 6},
    {'symbol': 'USDT', 'id': 23, 'decimals': 6},
    ]


state = chain.api.omnipool.asset_state(0)
print(state)
print(state.hub_reserve)
#Asset: 0
#	Reserve: 0
#	LRNA: 20220634396578953
#	Shares: 40600077930826935893
#	Protocol: 3694884494247885459

tokens = {}

asset_list = ['USDT','DOT','HDX']
id_list = [i for i in range(len(metadata)) if metadata[i]['symbol'] in asset_list]

for i in id_list:
    try:
        state = chain.api.omnipool.asset_state(i)
    except:
        continue
    tokens[metadata[i]['symbol']] = {
        'liquidity': state.reserve / 10**metadata[i]['decimals'],
        'LRNA': state.hub_reserve / 10**12
    }
chain.close()

pprint(tokens)

# print(tokens)

# get Omnipool data
# tokens = {
#     'USDT': {
#         'liquidity': 2062772,
#         'LRNA': 2062772
#     },
#     'DOT': {
#         # 'liquidity': 389000,
#         'liquidity': 350000,
#         'LRNA': 1456248
#     },
#     'HDX': {
#         'liquidity': 108000000,
#         'LRNA': 494896
#     }
# }

Asset: 0
	Reserve: 113517610038565580600
	LRNA: 20271952385389038
	Shares: 40600077930826935893
	Protocol: 3694884494247885459
20271952385389038
{'DOT': {'LRNA': 64552.8364994807, 'liquidity': 386091.3319116305},
 'HDX': {'LRNA': 20271.95238538904, 'liquidity': 113517610.03856558},
 'USDT': {'LRNA': 86561.65082493056, 'liquidity': 2121724.572242}}


In [70]:

# dot_usdt_order_book = {
#     'bids': [{'price': 3.60, 'amount': 200},
#              {'price': 3.59, 'amount': 100},
#              {'price': 3.50, 'amount': 100},
#              {'price': 3.40, 'amount': 2000}],
#     'asks': [{'price': 3.70, 'amount': 100},
#              {'price': 3.74, 'amount': 5000},
#              {'price': 3.80, 'amount': 200},
#              {'price': 3.90, 'amount': 2000}]
# }
# 
# hdx_usdt_order_book = {
#     'bids': [{'price': 0.03, 'amount': 2000},
#              {'price': 0.025, 'amount': 2000},
#              {'price': 0.02, 'amount': 2000},
#              {'price': 0.015, 'amount': 2000}],
#     'asks': [{'price': 0.04, 'amount': 2000},
#              {'price': 0.05, 'amount': 2000},
#              {'price': 0.06, 'amount': 2000},
#              {'price': 0.07, 'amount': 2000}]
# }

# hdx_dot_order_book = {
#     'bids': [{'price': 0.005, 'amount': 2000},
#              {'price': 0.004, 'amount': 2000}],
#     'asks': [{'price': 0.0052, 'amount': 2000},
#              {'price': 0.0055, 'amount': 2000}]
# }

dot_usdt_order_book = parse_kraken_orderbook(dotusd_orderbook)
hdx_usdt_order_book = parse_kraken_orderbook(hdxusd_orderbook)

dot_usdt_order_book_obj = OrderBook(
    bids=[[bid['price'], bid['amount']] for bid in dot_usdt_order_book['bids']],
    asks=[[ask['price'], ask['amount']] for ask in dot_usdt_order_book['asks']]
)

hdx_usdt_order_book_obj = OrderBook(
    bids=[[bid['price'], bid['amount']] for bid in hdx_usdt_order_book['bids']],
    asks=[[ask['price'], ask['amount']] for ask in hdx_usdt_order_book['asks']]
)

order_book = {
    ('DOT','USDT'): dot_usdt_order_book_obj,
    ('HDX','USDT'): hdx_usdt_order_book_obj,
    # ('HDX','DOT'): hdx_dot_order_book
}



lrna_fee = 0.0005
asset_fee = 0.0025
cex_fee = 0.0016
buffer = 0.0002


cex = CentralizedMarket(
    order_book=order_book,
    asset_list=asset_list,
    trade_fee=cex_fee
)

op_state = OmnipoolState(
    tokens=tokens,
    lrna_fee=lrna_fee,
    asset_fee=asset_fee,
    preferred_stablecoin='USDT',
)

init_state = op_state.copy()


In [71]:
all_swaps = get_arb_swaps(op_state, cex, buffer=buffer)
print(all_swaps)

{('DOT', 'USDT'): [], ('HDX', 'USDT'): []}


In [72]:
state = init_state.copy()
init_agent = Agent(holdings={'USDT': 1000000000, 'DOT': 1000000000, 'HDX': 1000000000}, unique_id='bot')
agent = init_agent.copy()
execute_arb(state, cex, agent, all_swaps)

profit = calculate_profit(init_agent, agent)

print(profit)
            
# for tkn_pair in all_swaps:
#     print(tkn_pair)
#     op_spot = OmnipoolState.price(state, tkn_pair[0], tkn_pair[1])
#     buy_spot = op_spot / ((1 - lrna_fee)*(1 - asset_fee))
#     sell_spot = op_spot * (1 - lrna_fee) * (1 - asset_fee)
#     print((sell_spot, op_spot, buy_spot))
    


{'USDT': 0, 'DOT': 0, 'HDX': 0}
